#### Project CCF pySpark RDD

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
from pyspark import SparkConf,SparkContext
import os

conf = SparkConf()
conf.setMaster('local')
conf.setAppName('pyspark-shell')
conf.set('spark.driver.host', '127.0.0.1')
conf.set("spark.ui.proxyBase", "")
conf.set("spark.cores.max","4")
conf.set("spark.executor.cores","2")
os.environ['PYSPARK_PYTHON'] = '/Users/ccompain/.pyenv/versions/miniconda3-latest/bin/python' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter' 
sc = SparkContext(conf=conf)

In [4]:
def CCF_dedup(data):
  dedup = data.map(lambda x: ((x[0],x[1]), 1))\
              .reduceByKey(lambda x,y: 1)\
              .map(lambda x: (x[0][0], x[0][1]))
#              .sortBy(lambda x:x[0])
  return dedup

def CCF_Iterate_map(pair):
    #map
    pair2=pair.map(lambda x: (x[1], x[0]))
    map_pair=pair.union(pair2) 
    #map_pair=pair.union(pair2).sortByKey(ascending=True)
    return map_pair

def f(x): return x
    
def CCF_Iterate_reduce(data):
    #find min value per key
    #couple (key, min)
    key_min=data.reduceByKey(lambda x,y: min(x,y)).filter(lambda x: x[0]>x[1])
    #put together: in one command
    valuelist_min=key_min.join(data).map(lambda x: (x[0],x[1][1])).cogroup(key_min).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
    
    #make RDD : (min, (list of value))
    min_valuelist=valuelist_min.map(lambda x:(x[1][1][0], x[1][0]))

    #use flatmapvalue to transform to RDD (min, value), then filter min != value, map to couple (value, min)

    min_value=min_valuelist.flatMapValues(f).filter(lambda x: x[0]!=x[1]).map(lambda x: (x[1],x[0]))
    countnewpair=min_value.count()

    #union couple (key, min) and (value, min)
    #unionkeyminvalue=key_min.union(min_value).sortBy(lambda x:x[1])
    unionkeyminvalue=key_min.union(min_value)

    return unionkeyminvalue
  

In [3]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")])
r.collect()

[('A', 'B'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('F', 'G'), ('G', 'H')]

In [5]:
# Initialize flags
new_pair_flag = True
iteration = 0
new_reduce = r
current_size = new_reduce.count()

In [7]:
print("################################")
print(" Start CCF RDD")
print("--------------------------------")
print(f"Number of pairs :{current_size}")
while new_pair_flag:
    iteration +=1
    print(f"*** Iteration {iteration} ***")
    new_map = CCF_Iterate_map(new_reduce)
    new_reduce_tmp = CCF_Iterate_reduce(new_map)
    new_size = new_reduce_tmp.count()
    print(f"--> Iteration {iteration} : Number of new pairs = {new_size-current_size}")
    if (new_size-current_size)>0:
        new_reduce = CCF_dedup(new_reduce_tmp)
    else:
        print("*** Stop the loop ***")
        print("Clean the last RDD of duplicate pairs")
        new_reduce = CCF_dedup(new_reduce_tmp)
        #print(f"Save last RDD in {output_directory}")
        #new_reduce.coalesce(1).saveAsTextFile(output_directory)
        new_pair_flag = False
print("--------------------------------")
print("Total iterations :",iteration)
#print("Results dump : ")
#subprocess.call(["hdfs", "dfs", "-ls", output_directory])
print("################################")


################################
 Start CCF RDD
--------------------------------
Number of pairs :6
*** Iteration 1 ***
--> Iteration 1 : Number of new pairs = 4
*** Iteration 2 ***
--> Iteration 2 : Number of new pairs = 9
*** Iteration 3 ***
--> Iteration 3 : Number of new pairs = 4
*** Iteration 4 ***
--> Iteration 4 : Number of new pairs = 0
*** Stop the loop ***
Clean the last RDD of duplicate pairs
--------------------------------
Total iterations : 4
################################


In [8]:
new_reduce.collect()

[('G', 'F'), ('E', 'A'), ('H', 'F'), ('D', 'A'), ('B', 'A'), ('C', 'A')]